In [459]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor

## **Exercise: Logistic Regression**

**Gunakan dataset 'titanic' dari seaborn.**

    - Target = survived
    - Fitur = pclass, sex, age, fare

- Isi missing value jika ada
- Lakukan modeling dengan Logistic Regression
- Cek multicollinearity
- Buat intepretasi hasil summary

**Info mengenai dataset dapat dilihat pada link berikut:** <https://www.kaggle.com/c/titanic/data>

In [460]:
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


In [461]:
df = df[['survived','pclass','sex','age','fare']]
df

,survived,pclass,sex,age,fare
0,0,3,male,22.0,7.2500
1,1,1,female,38.0,71.2833
2,1,3,female,26.0,7.9250
3,1,1,female,35.0,53.1000
4,0,3,male,35.0,8.0500
...,...,...,...,...,...
886,0,2,male,27.0,13.0000
887,1,1,female,19.0,30.0000
888,0,3,female,NaN,23.4500
889,1,1,male,26.0,30.0000


In [462]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    object 
 3   age       714 non-null    float64
 4   fare      891 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 34.9+ KB


In [463]:
df['sex'].replace({'male': 1, 'female': 0}, inplace=True)
df['sex'].value_counts()

sex
1    577
0    314
Name: count, dtype: int64

In [464]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   survived  891 non-null    int64  
 1   pclass    891 non-null    int64  
 2   sex       891 non-null    int64  
 3   age       714 non-null    float64
 4   fare      891 non-null    float64
dtypes: float64(2), int64(3)
memory usage: 34.9 KB


In [465]:
df.isna().sum()

survived      0
pclass        0
sex           0
age         177
fare          0
dtype: int64

In [466]:
from scipy.stats import shapiro
if shapiro(df['age'].dropna())[1] <0.05:
    print(f"pvalue = {shapiro(df['age'].dropna())[1]}, Data is not normaly distributed, therefore fill NaN with median")
    df['age'].fillna(df['age'].median(),inplace=True)
else:
    print(f"pvalue = {shapiro(df['age'].dropna())[1]}, Data is normaly distributed, therefore fill NaN with mean")
    df['age'].fillna(df['age'].mean(),inplace=True)

pvalue = 7.340329943872348e-08, Data is not normaly distributed, therefore fill NaN with median


In [467]:
df['age'].isna().sum()

0

In [468]:
df.duplicated().sum()

128

In [469]:
df.drop_duplicates(inplace=True)

In [470]:
df.duplicated().sum()

0

In [471]:
df

,survived,pclass,sex,age,fare
0,0,3,1,22.0,7.2500
1,1,1,0,38.0,71.2833
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500
...,...,...,...,...,...
885,0,3,0,39.0,29.1250
887,1,1,0,19.0,30.0000
888,0,3,0,28.0,23.4500
889,1,1,1,26.0,30.0000


In [472]:
dfo = df.copy()

for column in ['age','fare']:
    if pd.api.types.is_numeric_dtype(dfo[column]):
        Q1 = dfo[column].quantile(0.25)
        Q3 = dfo[column].quantile(0.75)
        IQR = Q3 - Q1     
        outliers = (dfo[column] < Q1 - 1.5 * IQR) | (dfo[column] > Q3 + 1.5 * IQR)
        dfo = dfo[~outliers]
    else:
        pass

dfo

,survived,pclass,sex,age,fare
0,0,3,1,22.0,7.2500
2,1,3,0,26.0,7.9250
3,1,1,0,35.0,53.1000
4,0,3,1,35.0,8.0500
5,0,3,1,28.0,8.4583
...,...,...,...,...,...
885,0,3,0,39.0,29.1250
887,1,1,0,19.0,30.0000
888,0,3,0,28.0,23.4500
889,1,1,1,26.0,30.0000


In [473]:
dfo.reset_index(inplace=True)
dfo.drop(columns='index',inplace=True)
dfo

,survived,pclass,sex,age,fare
0,0,3,1,22.0,7.2500
1,1,3,0,26.0,7.9250
2,1,1,0,35.0,53.1000
3,0,3,1,35.0,8.0500
4,0,3,1,28.0,8.4583
...,...,...,...,...,...
634,0,3,0,39.0,29.1250
635,1,1,0,19.0,30.0000
636,0,3,0,28.0,23.4500
637,1,1,1,26.0,30.0000


In [474]:
Y = dfo['survived']
Y

0      0
1      1
2      1
3      0
4      0
      ..
634    0
635    1
636    0
637    1
638    0
Name: survived, Length: 639, dtype: int64

In [475]:
X = dfo.drop(columns='survived')
X

,pclass,sex,age,fare
0,3,1,22.0,7.2500
1,3,0,26.0,7.9250
2,1,0,35.0,53.1000
3,3,1,35.0,8.0500
4,3,1,28.0,8.4583
...,...,...,...,...
634,3,0,39.0,29.1250
635,1,0,19.0,30.0000
636,3,0,28.0,23.4500
637,1,1,26.0,30.0000


# Multicollinearity

In [476]:
def cal_VIF(X):
    LVIF = [variance_inflation_factor(X.values,i) for i in range(len(X.columns))]
    VIF = pd.DataFrame({'feature':X.columns,'VIF':LVIF})
    return VIF

dcVIF=[]
while cal_VIF(X.drop(columns=dcVIF))['VIF'].max() > 4:
    dcVIF.append((cal_VIF(X.drop(columns=dcVIF))[cal_VIF(X.drop(columns=dcVIF))['VIF']==cal_VIF(X.drop(columns=dcVIF))['VIF'].max()]).iloc[0]['feature'])
    X.drop(columns=dcVIF)

In [477]:
X.drop(columns=dcVIF,inplace=True)
X

,pclass,sex,fare
0,3,1,7.2500
1,3,0,7.9250
2,1,0,53.1000
3,3,1,8.0500
4,3,1,8.4583
...,...,...,...
634,3,0,29.1250
635,1,0,30.0000
636,3,0,23.4500
637,1,1,30.0000


In [478]:
X = sm.add_constant(X)
X

,const,pclass,sex,fare
0,1.0,3,1,7.2500
1,1.0,3,0,7.9250
2,1.0,1,0,53.1000
3,1.0,3,1,8.0500
4,1.0,3,1,8.4583
...,...,...,...,...
634,1.0,3,0,29.1250
635,1.0,1,0,30.0000
636,1.0,3,0,23.4500
637,1.0,1,1,30.0000


# Modeling

In [479]:
model = sm.Logit(Y,X).fit()

Optimization terminated successfully.
         Current function value: 0.515784
         Iterations 6


In [480]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:               survived   No. Observations:                  639
Model:                          Logit   Df Residuals:                      635
Method:                           MLE   Df Model:                            3
Date:                Tue, 22 Aug 2023   Pseudo R-squ.:                  0.2198
Time:                        15:36:49   Log-Likelihood:                -329.59
converged:                       True   LL-Null:                       -422.42
Covariance Type:            nonrobust   LLR p-value:                 5.273e-40
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          3.0111      0.542      5.556      0.000       1.949       4.073
pclass        -0.9043      0.158     -5.736      0.000      -1.213      -0.595
sex           -2.2075      0.203    -10.859      0.000      -2.606      -1.809
fare          -0.0010      0.008     -0.127      0.899      -0.017       0.015
==============================================================================
"""

# Accuracy

In [481]:
predict = model.predict(X)
predict

0      0.128204
1      0.571963
2      0.886059
3      0.128111
4      0.128063
         ...   
634    0.566509
635    0.888483
636    0.567971
637    0.467007
638    0.128146
Length: 639, dtype: float64

In [482]:
predict = np.where(predict>0.5,1,0)

In [483]:
df_survived = pd.DataFrame({'survived':Y,'predict':predict})
df_survived

,survived,predict
0,0,0
1,1,1
2,1,1
3,0,0
4,0,0
...,...,...
634,0,1
635,1,1
636,0,1
637,1,0


In [484]:
from sklearn.metrics import accuracy_score
print(f" Model ini dapat memprediksi survival rate sebesar {round(accuracy_score(df_survived['survived'],df_survived['predict'])*100)} % .")

 Model ini dapat memprediksi survival rate sebesar 75 % .


# Log-Likelihood Ratio Test

In [485]:
if model.llr_pvalue < 0.05:
    print(f'LLR p-value = {model.llr_pvalue}, (reject Ho): Minimal 1 feature berepengaruh signifikan terhadap target.')
else:
    print(f'LLR p-value = {model.llr_pvalue}, (accept Ho): Semua feature tidak berepengaruh signifikan  terhadap target.')

LLR p-value = 5.273054622229002e-40, (reject Ho): Minimal 1 feature berepengaruh signifikan terhadap target.


# Wald Test

In [486]:
for i in model.pvalues.index:
    if model.pvalues[i] < 0.05:
        print(f'p-value = {model.pvalues[i]}, (reject Ho): Feature ({i}) dibutuhkan dan berepengaruh signifikan terhadap target.')
    else:
        print(f'p-value = {model.pvalues[i]}, (accept Ho): Feature ({i}) tidak dibutuhkan dan tidak berepengaruh signifikan terhadap target.')

p-value = 2.7671603273727786e-08, (reject Ho): Feature (const) dibutuhkan dan berepengaruh signifikan terhadap target.
p-value = 9.677258507614842e-09, (reject Ho): Feature (pclass) dibutuhkan dan berepengaruh signifikan terhadap target.
p-value = 1.8056962572342123e-27, (reject Ho): Feature (sex) dibutuhkan dan berepengaruh signifikan terhadap target.
p-value = 0.8987526621544712, (accept Ho): Feature (fare) tidak dibutuhkan dan tidak berepengaruh signifikan terhadap target.


# Odds Ratio

In [487]:
model.params

const     3.011125
pclass   -0.904316
sex      -2.207501
fare     -0.001049
dtype: float64

In [488]:
dfo.describe().loc[['min','max'],model.params.index[1:]]

,pclass,sex,fare
min,1.0,0.0,0.0
max,3.0,1.0,71.0


In [489]:
model.params

const     3.011125
pclass   -0.904316
sex      -2.207501
fare     -0.001049
dtype: float64

In [490]:
for i in model.params.index:
    if i=='const':
        if model.params[0]>1:
            print(f'Coefficient Beta(0) = {model.params[0]}, semakin besar Beta(0) maka semakin besar peluang untuk survived')
        elif model.params[0]<1:
            print(f'Coefficient Beta(0) = {model.params[0]}, semakin besar Beta(0) maka semakin kecil peluang untuk survived')
        else:
            print(f'Coefficient Beta(0) = {model.params[0]},  Beta(0) tidak mempengaruhi peluang untuk survived')

Coefficient Beta(0) = 3.0111250805629335, semakin besar Beta(0) maka semakin besar peluang untuk survived


In [491]:
OR_pclass = round(np.exp(model.params[1]*(dfo['pclass'].max()-dfo['pclass'].min()))*100)
OR_pclass
print(f'Penumpang kelas 3 berpeluang survived sebesar {OR_pclass} % dibanding Kelas 1')

Penumpang kelas 3 berpeluang survived sebesar 16 % dibanding Kelas 1


In [492]:
OR_sex = round(np.exp(model.params[2]*(dfo['sex'].max()-dfo['sex'].min()))*100)
OR_sex
print(f'Pria berpeluang survived sebesar {OR_sex} % dibanding Perempuan')

Pria berpeluang survived sebesar 11 % dibanding Perempuan


In [493]:
OR_fare = round(np.exp(model.params[3]*(dfo['fare'].max()-dfo['fare'].min()))*100)
OR_fare
print(f'Penumpang dengan fare tertinggi berpeluang survived sebesar {OR_fare} % dibanding penumpang dengan fare terendah')

Penumpang dengan fare tertinggi berpeluang survived sebesar 93 % dibanding penumpang dengan fare terendah
